# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix
from definitions import *

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)
catalog.shape

(68, 199)

In [3]:
xmm_catalog = pd.read_csv("data/gevtevsimbadxmm.txt", sep='\t', index_col=0)
xmm_catalog.shape

(67, 238)

In [4]:
xmm_catalog[list_xmm_spectra_columns()]

,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
11,0.000000e+00,0.000000e+00,2.872300e-15,0.000000e+00,3.058640e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14
35,4.363840e-16,2.649270e-15,5.332780e-15,1.147440e-14,2.130470e-14


In [5]:
# columns contain spectrums in xmm
xmm_spectrum_columns = list_xmm_spectra_columns()

In [6]:
catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
catalog

C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,NaN,NaN,NaN,NaN,NaN
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,6.310590e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15,NaN,NaN,NaN,NaN,NaN
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,NaN,NaN,NaN,NaN,NaN
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,NaN,NaN,NaN,NaN,NaN
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,NaN,NaN,NaN,NaN,NaN
5,85,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
6,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,NaN,NaN,NaN,NaN,NaN
7,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
8,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN
9,11,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,3.033065e-11,5.807977e-13,5.926255e-14,1.818965e-15,2.823374e-16,NaN,NaN,NaN,NaN,NaN


Columns contained spectrum in gev:

In [7]:
gev_spectrum_columns = list_gev_spectrum_columns()

Columns contained spectrum in simbad:

In [8]:
s_spectrum_columns = list_s_spectrum_columns()

Columns contained spectrum in tev:

In [9]:
tev_spectrum_columns = list_tev_spectrum_columns()

In [10]:
s_other_columns = list_s_other_columns()

In [11]:
gev_other_columns = list_gev_other_columns()

In [12]:
tev_other_columns = list_tev_other_columns()

In [13]:
xmm_other_columns = list_xmm_other_columns()

In [14]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
spectrum_with_xmm_columns = spectrum_columns + xmm_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [15]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [16]:
#spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [17]:
#for i in tev_spectrum_columns:
#    if len(re.findall("\d+", i)) == 2:
#        E = 0.3
#    else:
#        E = int(re.findall("\d+", i)[0])
#    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [18]:
spectrums = spectrums.fillna(0)

In [19]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000,0.000,0.000000,0.00,...,5.610705e-15,6.128219e-17,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,0.0,0.000000,11.160,10.510000,0.00,...,4.356098e-14,2.865129e-15,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000,0.000,0.000000,0.00,...,5.610705e-15,6.128219e-17,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000,0.000,0.000000,0.00,...,1.856180e-15,3.184437e-16,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000,0.000,0.000000,0.00,...,5.610705e-15,6.128219e-17,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000,0.000,0.000000,0.00,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000,0.000,0.000000,0.00,...,1.856180e-15,3.184437e-16,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,13.120000,13.080,11.950000,0.00,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,12.390000,12.010,11.300000,11.05,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,11.270000,11.610,10.750000,10.19,...,1.818965e-15,2.823374e-16,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


Normalization:

In [20]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [21]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,7.465491e-14,1.892097e-12,1.148535e-13,7.886758e-13,1.822269e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,1.425665e-16,1.557164e-18,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,3.485933e-13,3.483344e-13,2.104312e-14,3.777814e-13,1.256978e-13,0.0,0.000000,0.185392,0.174594,0.000000,...,7.236428e-16,4.759604e-17,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,8.213710e-14,2.081730e-12,1.263645e-13,8.677197e-13,2.004903e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,1.568551e-16,1.713229e-18,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,1.336783e-13,1.514559e-12,1.461508e-14,8.303320e-13,9.620438e-13,0.0,0.000000,0.000000,0.000000,0.000000,...,5.271741e-17,9.044128e-18,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,1.175218e-13,2.978540e-12,1.808024e-13,1.241533e-12,2.868615e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,2.244282e-16,2.451288e-18,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,1.420025e-13,1.608871e-12,1.552516e-14,8.820367e-13,1.021950e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,1.420025e-13,1.608871e-12,1.552516e-14,8.820367e-13,1.021950e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,5.600012e-17,9.607305e-18,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,3.678921e-14,5.947283e-13,1.834295e-13,4.169087e-13,4.098584e-13,0.0,0.169616,0.169099,0.154490,0.000000,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,3.364506e-14,5.439005e-13,1.677529e-13,3.812781e-13,3.748303e-13,0.0,0.146489,0.141996,0.133602,0.130646,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,6.697221e-14,9.986226e-13,1.529252e-12,4.581479e-13,1.488680e-12,0.0,0.127302,0.131143,0.121429,0.115103,...,2.054648e-17,3.189198e-18,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


In [22]:
# spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [23]:
def normalize_spectra(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    
    spectrum_columns =  s_spectrum_columns +gev_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    
    display(spectrums)
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized.txt", index=False, sep='\t')

In [24]:
def calculate_spectra_old(catalog_name, s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns,
                      s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    
    spectrum_columns =  s_spectrum_columns +gev_spectrum_columns + tev_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    display(spectrums)
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    spectrums.to_csv("markeddata/" + catalog_name + "_normalized.txt", index=False, sep='\t')

In [25]:
def calculate_spectra(catalog_name, xmm_catalog_name, output_catalog_name, 
                      s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns, xmm_spectrum_columns,
                      spectrum_columns, s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    xmm_catalog = pd.read_csv("data/{}.txt".format(xmm_catalog_name), sep='\t', index_col=0)
    catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns + xmm_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums[spectrum_columns].sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    spectrums.to_csv("markeddata/" + output_catalog_name + "_normalized.txt", index=False, sep='\t')

* adding xmm spectrum columns to the gevtevsimbad, tevsimbad and gevsimbad spectrums

In [26]:
calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
                  xmm_catalog_name="gevtevsimbadxmm",
                  output_catalog_name="gevtevsimbadxmm_spectrum",
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns,
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
                  xmm_catalog_name="gevtevsimbadxmm",
                  output_catalog_name="gevtevsimbadxmm_spectrum",
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns,
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="tevsimbad_TeVspectrum",
                  xmm_catalog_name="tevsimbadxmm",
                  output_catalog_name="tevsimbadxmm_spectrum",
                  s_spectrum_columns =s_spectrum_columns, 
                  gev_spectrum_columns=[], 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=[],
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevsimbad", 
                  xmm_catalog_name="gevsimbadxmm",
                  output_catalog_name="gevsimbadxmm_spectrum",
                  s_spectrum_columns=s_spectrum_columns, 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=[], 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=[])

C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
1,0.140281,0.140177,0.008468,0.152027,0.050583,0.0,0.421867,0.080695,0.005592,0.000291,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b'PWN ',b'',b'unid',0
2,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
3,0.034047,0.385752,0.003722,0.211482,0.245029,0.0,0.119148,0.000602,0.000201,0.000013,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
5,0.038689,0.438338,0.004230,0.240312,0.278431,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,0.034047,0.385752,0.003722,0.211482,0.245029,0.0,0.119148,0.000602,0.000201,0.000013,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,0.013691,0.204153,0.312632,0.093661,0.304338,0.0,0.070041,0.001341,0.000137,0.000004,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
1,0.140281,0.140177,0.008468,0.152027,0.050583,0.0,0.421867,0.080695,0.005592,0.000291,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b'PWN ',b'',b'unid',0
2,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
3,0.034047,0.385752,0.003722,0.211482,0.245029,0.0,0.119148,0.000602,0.000201,0.000013,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,0.013311,0.337365,0.020479,0.140623,0.324915,0.0,0.151398,0.011056,0.000829,0.000025,...,0.00000,0.000000,0.000000,0.00000,0.000000,b' ',b' ',b'',b'unid',0
5,0.038689,0.438338,0.004230,0.240312,0.278431,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,0.034047,0.385752,0.003722,0.211482,0.245029,0.0,0.119148,0.000602,0.000201,0.000013,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,0.022409,0.362261,0.111730,0.253947,0.249653,0.0,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,0.013691,0.204153,0.312632,0.093661,0.304338,0.0,0.070041,0.001341,0.000137,0.000004,...,0.00000,0.000000,0.000000,0.00000,0.000000,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,s_FLUX_I,s_FLUX_G,s_FLUX_J,s_FLUX_H,s_FLUX_K,s_FLUX_u,...,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,tev_fermi_names,tev_classes,marked
0,0.000000,0.000000,0.000000,0.215163,0.000000,0.210133,0.195761,0.192014,0.186929,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'UCAC4 105-013663',Be,False,b'',b'other',0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.385240,0.332180,0.282580,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'MSX6C G331.5487-00.5364',Be,False,b'',b'unid',0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.395248,0.332867,0.271884,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'SSTGC 519103',Be,False,b'',b'unid',0
3,0.000000,0.208395,0.204788,0.000000,0.000000,0.000000,0.197245,0.195442,0.194130,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'RMC 141',B,False,b'',b'pwn',0
4,0.000000,0.124084,0.123543,0.125641,0.124485,0.123546,0.125231,0.126624,0.126846,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[AAA97b] J053648.7-691700',B,False,b'',b'other',0
5,0.000000,0.125546,0.124929,0.124851,0.122729,0.123609,0.126086,0.125932,0.126318,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'OGLE LMC-ECL-20994',B,False,b'',b'pwn',0
6,0.321967,0.338205,0.339828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[ST92] 2-62B',B,False,b'',b'other',0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.351673,0.329465,0.318862,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[KKM2013] 7',B,False,b'',"b'pwn,snr'",0
8,0.336570,0.345334,0.318096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'[ST92] 2-62',B,False,b'',b'other',0
9,0.119025,0.123655,0.118016,0.114094,0.098441,0.117688,0.105457,0.102760,0.100864,0.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'HTR 13',B,False,b'',b'pwn',0


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,marked
0,1.656470e-02,3.181164e-01,1.167143e-01,1.949821e-01,3.536225e-01,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,b'[KRL2007b] 194',Be,True,b'1FGL J1636.4-4737c',b'snr ',0
1,9.659359e-15,1.778570e-13,3.858887e-14,8.784309e-14,1.763382e-13,0.0,0.098352,0.101852,0.098629,0.097708,...,0.0,0.0,0.0,0.0,b'HD 328990',Be,False,b'1FGL J1651.5-4602c',b' ',0
2,2.885774e-14,1.754809e-13,1.940037e-13,1.183821e-13,1.661737e-13,0.0,0.126725,0.129164,0.121034,0.115073,...,0.0,0.0,0.0,0.0,b'UCAC2 5051621',Be,False,b' ',b'spp ',0
3,3.989508e-13,9.114596e-13,3.263733e-12,8.938205e-13,1.391035e-12,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,b'SSTGC 519103',Be,False,b'1FGL J1745.6-2900c',b'spp ',0
4,1.298479e-14,6.494359e-14,7.579482e-14,4.062574e-14,8.512149e-14,0.0,0.000000,0.147016,0.147337,0.000000,...,0.0,0.0,0.0,0.0,b'LIN 341',Be,False,b'1FGL J0101.3-7257 ',b'GAL ',0
5,5.193645e-15,2.181423e-14,5.532339e-14,1.368890e-14,3.013832e-14,0.0,0.140946,0.149603,0.147168,0.000000,...,0.0,0.0,0.0,0.0,b'LS VI -04 15',B,False,b' ',b' ',0
6,1.400598e-15,2.332674e-14,7.971940e-14,9.706428e-15,3.455149e-14,0.0,0.000000,0.182442,0.177006,0.000000,...,0.0,0.0,0.0,0.0,b'LS I +56 58',B,False,b' ',b' ',0
7,2.393573e-19,2.189840e-15,6.887979e-15,1.238084e-15,6.179302e-15,0.0,0.000000,0.085799,0.083993,0.084840,...,0.0,0.0,0.0,0.0,b'PG 1101+529',B,False,b' ',b' ',0
8,7.356776e-18,1.544090e-13,2.159914e-13,5.344441e-14,2.087504e-13,0.0,0.000000,0.000000,1.000000,0.000000,...,0.0,0.0,0.0,0.0,b'GALEX J175340.5-500741',B,False,b'1FGL J1754.0-5002 ',b'bcu ',0
9,3.089607e-14,1.878757e-13,2.077069e-13,1.267439e-13,1.779111e-13,0.0,0.120687,0.120590,0.115078,0.112370,...,0.0,0.0,0.0,0.0,b'HD 306182',B,False,b' ',b'spp ',0


* contain objects, which have spectrum information from catalogs (gev, tev, simbad) and are in their intersection, but were not found in xmm catalog

In [27]:
normalize_spectra(catalog_name="pd_gevtevxmm_simbadclasses", 
                   s_spectrum_columns=xmm_spectrum_columns, 
                   gev_spectrum_columns=gev_spectrum_columns, 
                   tev_spectrum_columns=tev_spectrum_columns, 
                   s_other_columns=xmm_other_columns,
                   gev_other_columns=gev_other_columns,
                   tev_other_columns=tev_other_columns)

,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,...,tev_3TeV,tev_10TeV,tev_30TeV,xmm_IAUNAME,xmm_WEBPAGE_URL,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,8.307141e-05,2.263321e-03,5.500074e-03,2.183875e-03,3.445482e-05,0.053683,0.056830,2.645774e-03,0.029766,0.113095,...,0.000348,0.000288,2.427262e-04,b'3XMM J002515.0+640847',b'http://xmm-catalog.irap.omp.eu/source/200962...,b' ',b'snr ',b'',b'snr',0
1,5.443995e-06,1.818843e-05,3.900402e-03,2.315009e-03,5.748870e-06,0.053890,0.057050,2.655983e-03,0.029880,0.113532,...,0.000349,0.000289,2.436628e-04,b'3XMM J002515.3+640928',b'http://xmm-catalog.irap.omp.eu/source/200962...,b' ',b'snr ',b'',b'snr',0
2,3.307245e-04,3.220516e-03,5.539306e-03,1.357200e-03,0.000000e+00,0.053662,0.056808,2.644750e-03,0.029754,0.113051,...,0.000348,0.000288,2.426323e-04,b'3XMM J002516.3+640834',b'http://xmm-catalog.irap.omp.eu/source/205111...,b' ',b'snr ',b'',b'snr',0
3,5.168827e-05,1.657308e-03,2.456047e-03,3.619529e-04,3.076459e-06,0.053983,0.057148,2.660566e-03,0.029932,0.113727,...,0.000350,0.000290,2.440833e-04,b'3XMM J002516.4+640756',b'http://xmm-catalog.irap.omp.eu/source/200962...,b' ',b'snr ',b'',b'snr',0
4,1.080182e-04,2.311462e-03,1.207137e-04,0.000000e+00,6.704592e-04,0.054054,0.057224,2.664093e-03,0.029972,0.113878,...,0.000350,0.000290,2.444068e-04,b'3XMM J002516.8+641108',b'http://xmm-catalog.irap.omp.eu/source/204123...,b' ',b'snr ',b'',b'snr',0
5,2.978488e-05,9.843300e-04,4.192689e-04,3.770413e-04,7.862385e-05,0.054126,0.057300,2.667625e-03,0.030011,0.114029,...,0.000351,0.000291,2.447308e-04,b'3XMM J002516.9+640658',b'http://xmm-catalog.irap.omp.eu/source/204123...,b' ',b'snr ',b'',b'snr',0
6,3.349050e-04,4.256873e-03,1.006476e-02,7.663504e-03,1.317959e-04,0.053011,0.056119,2.612667e-03,0.029393,0.111680,...,0.000344,0.000285,2.396890e-04,b'3XMM J002517.3+640454',b'http://xmm-catalog.irap.omp.eu/source/205111...,b' ',b'snr ',b'',b'snr',0
7,8.793109e-03,1.336494e-01,3.293432e-01,2.916743e-01,8.202143e-02,0.008379,0.008871,4.129778e-04,0.004646,0.017653,...,0.000054,0.000045,3.788704e-05,b'3XMM J002517.5+640506',b'http://xmm-catalog.irap.omp.eu/source/200962...,b' ',b'snr ',b'',b'snr',0
8,2.685506e-05,1.989122e-03,3.317248e-03,2.205353e-03,5.448056e-08,0.053820,0.056975,2.652525e-03,0.029842,0.113384,...,0.000349,0.000289,2.433456e-04,b'3XMM J002518.1+640708',b'http://xmm-catalog.irap.omp.eu/source/200962...,b' ',b'snr ',b'',b'snr',0
9,2.244221e-05,9.740841e-04,9.187210e-04,0.000000e+00,8.418247e-07,0.054125,0.057298,2.667553e-03,0.030011,0.114026,...,0.000351,0.000291,2.447242e-04,b'3XMM J002520.7+640616',b'http://xmm-catalog.irap.omp.eu/source/203105...,b' ',b'snr ',b'',b'snr',0


In [28]:
# calculate_spectra(catalog_name="gevtev_TeVspectrum", 
#                   s_spectrum_columns=[], 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=[],
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
#                   s_spectrum_columns =s_spectrum_columns, 
#                   gev_spectrum_columns=[], 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=[],
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="gevsimbad", 
#                   s_spectrum_columns=s_spectrum_columns, 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=[], 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=[])